# Import Packages And Functions

In [1]:
# Packages
import sys; sys.path.append('..')
from Code.data import data_load
import Code.influence as influence
import Code.block_edges as be
import Code.communities as communities
import networkx as nx

# Load The Dataset

In [ ]:
Graph_To_Be_Used = 'facebook'

path = '../data/edges_'+Graph_To_Be_Used+'.txt'  # dataset file path
init_rate = 0.001 # Initial fraction of seed nodes
k = 0.01 # Fraction of Edges to be Blocked


nodes, edges, weights = data_load(path)
graph = nx.Graph()
min_wt = min(weights.values())
eps = min_wt*0.001
wt_factor = 1
for e in edges:
    graph.add_edge(e[0],e[1], weight = weights[e]/wt_factor, distance = 1-weights[e]+eps)

print ("No of edges to be removed:", int(k*len(edges)))
print ('No of Initial Seeds:', int(init_rate*len(nodes)))
print ("Clustering Coeff: ",nx.average_clustering(graph))
print ("Average Deg: ", len(graph.edges)*2/len(graph.nodes))

# Average Number Of Nodes Infected Initially

In [ ]:
iters = 100 # Number of Iterations of The ICM
a = influence.influence_count_multiple(graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("Nodes Infected = ",a, "Percentage = ", b)

# RNDM Method

In [ ]:
output = be.RNDM(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges Randomly:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# HWT Method

In [ ]:
output = be.HWGT(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Weight:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# DEG Method

In [ ]:
output = be.HDEG(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Sum of Degrees of Adjacent Nodes:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# WDEG Method

In [ ]:
output = be.WDEG(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Sum of Weighted Degrees of Adjacent Nodes:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# CLO Method

In [ ]:
output = be.CLO(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Sum of Closeness of Adjacent Nodes:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# WCLO Method

In [ ]:
output = be.WCLO(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Sum of Weighted Closeness of Adjacent Nodes:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# BET Method

In [ ]:
output = be.BTWN(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Betweenness:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# WBET Method

In [ ]:
output = be.WBET(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Weighted Betweenness:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# PGRK Method

In [ ]:
output = be.PGRK(graph)
final_graph = be.rem_edges(graph.copy(),output,k)

a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges With Heighest Sum Of Pagerank of Adjacent Nodes:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# IEED Method

In [ ]:
edge_list = be.IEED_remove_edges(nodes.copy(),edges.copy(),weights,k)
final_graph = nx.Graph()
min_wt = min(weights.values())
eps = min_wt*0.001
wt_factor = 1
for e in edge_list:
    final_graph.add_edge(e[0],e[1], weight = weights[e]/wt_factor, distance = 1-weights[e]+eps)


a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges using IEED:")
print ("Nodes Infected = ",a, "Percentage = ", b)

# Community Method

## Community Detection And Edge Removal

In [ ]:
# Select the initial 'resolution parameter' and the 'increasing factor'. This needs to be done manually, and results will vary depending on the dataset.
resolution = 0.15
increasing_factor = 1.05

happened = 0
best_p_conn = 0
found = ([],[],[])

while happened <= 5:
    for i in range(5):
        comm = communities.detect_communities(graph,resolution)
        conn_edges,comm_edges, conn_nodes = communities.connecting_edges(comm,graph)
        p_conn = (len(conn_edges)/(len(edges))) # Percentage of connecting edges
        print ('fraction of connecting edges', p_conn)
        if (p_conn <=k) & (p_conn > best_p_conn):
            found = conn_edges
            best_p_conn = p_conn
        
    resolution *= increasing_factor
    if p_conn >k:
        happened += 1
    print ('Resolution: ',resolution)

final_graph = communities.graph_after_rem_edges(found,graph.copy())

## ICM After Edge Removal

In [ ]:
a = influence.influence_count_multiple(final_graph,init_rate,iters)
b = 100*(a/len(nodes))
print ("After Blocking Edges Connecting Different Communities:")
print ("Nodes Infected = ", a, "Percentage = ", b)